In [1]:
from analysis.analysis import *
import matplotlib.pyplot as plt
from glob import glob
from tqdm import tqdm


def plot_trajectories(trajectories, control_traj, dpi=200, savepath=None):
    fig, axs = plt.subplots(
        len(trajectories) + 1, 1, sharex=True, sharey=True, figsize=(5, 10)
    )

    control_traj.plot(x="time[hr]", y="x", ax=axs[0], color="orange")
    [
        ax.plot(
            (np.arange(len(traj)) * 0.00075 * 8 / 60)[::500],
            traj[::500],
            color="purple",
        )
        for ax, traj in zip(axs[1:], trajectories)
    ]
    plt.xlabel("Time (hr)")
    plt.ylabel(r"$x (\mu$m)")
    axs[0].set_title("Bistable")
    axs[0].legend(["Sim"], bbox_to_anchor=(1, 1))
    axs[1].legend(["Prediction"], bbox_to_anchor=(1, 1))
    if savepath is not None:
        plt.savefig("bs.png", dpi=dpi)
    plt.show()

In [2]:
gid = 2
control = bootstrap(
    read_fulltake(
        f"../_server/sim_data/defaults/parquets/fulltake_gid{gid}.parquet",
        scale_position=True,
    ),
    1,
)
predicted_trajs = np.load(f"trajs_{gid}.npy").reshape(-1, 1)

ValueError: cannot reshape array of size 39911408 into shape (2400,240000)

In [ ]:
to_hours = 0.00075 * 8 / 60  # -> hours

dwell_times = []
long_files = []
for f in tqdm(files, total=len(files)):
    df = pd.read_parquet(f)
    if df.shape[0] > 200000:
        long_files.append(f)
    if get_itinerary(df) is None:
        continue
    
    t = np.diff(get_itinerary(df), axis=1) * to_hours
    dwell_times.extend(list(t.ravel()))

In [ ]:
plt.figure(figsize=(3, 2), dpi=100)
plt.hist(
    np.load(f"../_server/sim_data/defaults/hopping times/fulltake_gid{gid}.npy"),
    density=True,
    bins=40,
    alpha=0.75,
    color="black",
    label="Train",
)
plt.hist(
    dwell_times,
    density=True,
    bins=40,
    alpha=0.75,
    color="salmon",
    label="Test",
)
plt.xlabel("Hopping times (hr)")
plt.legend()
plt.show()

In [ ]:
trajs = [pd.read_parquet(f).x.values for f in files]
plot_trajectories(trajs[:5], control)